In [1]:
import os
import sys
import torch

PUBDIR = os.getcwd()
ROOT_DIR = os.path.dirname(PUBDIR)
DATA_DIR = os.path.join(ROOT_DIR, "data")
sys.path.append(ROOT_DIR)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Load ontology
from core.data_model import Patient, Disease, Ontology
from core.data_model import Diseases
from core.io_ops import load_pickle

disease_data = load_pickle(os.path.join(DATA_DIR, "diseases.pickle"))
vectorized_hpo = load_pickle(os.path.join(DATA_DIR, "hpo_definition.vector.pickle"))
ontology = Ontology(vectorized_hpo)
omim_diseases = Diseases([disease for disease in disease_data if disease.id.startswith("OMIM")])

/home/heon/repositories/LaRa/core/data_model.py:283: UnicodeWarning: unsound encoding, assuming ISO-8859-1 (73% confidence)
  ontology = pronto.Ontology(SORUCE_URL["hpo_obo"])


### 1. Load benchmark patient dataset

In [3]:
# download public dataset from zeonodo
if not os.path.exists('../data/phenopackets.zip'):
    ! wget https://zenodo.org/records/3905420/files/phenopackets.zip?download=1
    ! mv phenopackets.zip?download=1 ../data/phenopackets.zip
    ! unzip -o ../data/phenopackets.zip -d ../data/

--2024-04-01 15:45:03--  https://zenodo.org/records/3905420/files/phenopackets.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 572349 (559K) [application/octet-stream]
Saving to: ‘phenopackets.zip?download=1’

phenopackets.zip?do 100%[===================>] 558.93K   495KB/s    in 1.1s    

2024-04-01 15:45:05 (495 KB/s) - ‘phenopackets.zip?download=1’ saved [572349/572349]

Archive:  ../data/phenopackets.zip
   creating: ../data/phenopackets/
  inflating: ../data/phenopackets/Naz_Villalba-2016-NLRP3-proband.json  
  inflating: ../data/phenopackets/Abdul_Wahab-2016-GCDH-Patient_5.json  
  inflating: ../data/phenopackets/Papanastasiou-2010-STAT3-12_year_old_girl.json  
  inflating: ../data/phenopackets/Prada-2014-CTSA-BAB3767.json  
  inflating: ../data/phenopackets/Luco-2016-DYRK1A-Patient_2.json  
  infl

In [4]:
import glob
from core.benchmark import load_phenopacket_patients
from core.data_model import HPO, HPOs, Patient, Patients

benchmark_patients:Patients = load_phenopacket_patients(
    phenopacket_dir=os.path.join(DATA_DIR, "phenopackets"),
    ontology=ontology
)
print(benchmark_patients)

Patients(N=384)


### Model 1: Phen2Disease

Download prerequsite file

In [5]:
if not os.path.exists("lin_similarity_matrix.json"):
    ! pip install gdown
    ! gdown 1CSYfDj5fG9SsosIDlG-hLAoKp9eMHxjH
    ! gunzip -f lin_similarity_matrix.json.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 17.9 MB/s eta 0:00:00
Downloading...
From (original): https://drive.google.com/uc?id=1CSYfDj5fG9SsosIDlG-hLAoKp9eMHxjH
From (redirected): https://drive.google.com/uc?id=1CSYfDj5fG9SsosIDlG-hLAoKp9eMHxjH&confirm=t&uuid=dde4d559-807f-404c-99d5-78b4b4f33a60
To: /home/heon/repositories/LaRa/publication/lin_similarity_matrix.json.gz
100%|████████████████████████████████████████| 625M

In [6]:
import tqdm
import numpy as np
from core.benchmark import get_phen2disease
from core.io_ops import read_json

phen2disease_result = np.zeros((len(benchmark_patients), len(omim_diseases)))
labels = np.zeros((len(benchmark_patients), len(omim_diseases)))
pheno2disease_sim_mat:dict = read_json("lin_similarity_matrix.json")
    
for patient_idx, patient in tqdm.tqdm(enumerate(benchmark_patients), total=len(benchmark_patients)):
    true_disease_indices = [
        disease_idx for disease_idx, disease 
        in enumerate(omim_diseases) 
        if disease.id in patient.disease_ids
    ]
    labels[patient_idx, true_disease_indices] = 1
    
    process = list()
    for disease_idx, disease in enumerate(omim_diseases):
        phen2disease_result[patient_idx, disease_idx] = get_phen2disease(patient, disease, pheno2disease_sim_mat)

np.save("phen2disease_result", phen2disease_result)

 17%|█▋        | 67/384 [01:38<05:55,  1.12s/it]

### Model 2: LaRA

load model

In [ ]:
import torch
from core.datasets import (
    StochasticPairwiseDataset,
    collate_for_stochastic_pairwise_eval,
)
from torch.utils.data import DataLoader
from core.networks import Transformer

lara = Transformer(**{
    "output_size": 128,
    "hidden_dim": 2048,
    "input_size": 1536,
    "n_layers": 32,
    "nhead": 32,
    "batch_first": False,
})
checkpoint:dict = torch.load(
    os.path.join(ROOT_DIR, "data", "epoch=115-step=185368.ckpt"),
    map_location=DEVICE
)
params = {
    k.replace('network.', ''): v 
    for k, v in checkpoint['state_dict'].items()
}
lara.load_state_dict(state_dict=params)
lara.eval()
lara.cuda(DEVICE)

build dataset for LaRA

In [ ]:
from core.augmentation import TruncateOrPad

benchmark_dataset = StochasticPairwiseDataset(
    benchmark_patients,
    disease_data,
    max_len=15,
)
benchmark_dataset.validate()

benchmark_dataloader = DataLoader(
    benchmark_dataset,
    batch_size=1,
    num_workers=0,
    collate_fn=collate_for_stochastic_pairwise_eval,
    shuffle=False,
    pin_memory=True,
)

Caching disease vectors

In [ ]:
import numpy as np
def cosine_sim(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

padder = TruncateOrPad(15, stochastic=False, weighted_sampling=True)
lara_disease_vectors = list()
with torch.no_grad():
    for disease in tqdm.tqdm(omim_diseases):
        disease_tensor:torch.Tensor = padder(
            torch.tensor(disease.hpos.vector, dtype=torch.float32, device=DEVICE), disease
        )
        target_vector = lara(disease_tensor).squeeze().cpu().numpy()
        lara_disease_vectors.append(target_vector)
        
lara_disease_vectors = np.stack(lara_disease_vectors, axis=0)

Forward & Calculate disease similarity b/t patient and known rare diseaes.

In [ ]:
lara_result = np.zeros((len(benchmark_patients), len(omim_diseases)))

with torch.no_grad():
    for patient_idx, paitent in tqdm.tqdm(enumerate(benchmark_patients)):
        for disease_idx, disease in enumerate(omim_diseases):
            input_src = padder(
                torch.tensor(patient.hpos.vector, dtype=torch.float32, device=DEVICE), patient
            )
            patient_vector = lara(input_src).squeeze().cpu().numpy()
            
            lara_result[patient_idx, disease_idx] = cosine_sim(
                patient_vector,
                lara_disease_vectors[disease_idx]
            )

### Model 3: Node level Semantic similarity

Install SemanticSimilarity (Node level similarity)

In [ ]:
if not os.path.exists('SemanticSimilarity'):
    ! git clone https://github.com/4pygmalion/SemanticSimilarity.git
    ! cd SemanticSimilarity
    ! python3 -m pip install . 

Load calculator

In [ ]:
from semantic_similarity.calculator import NodeLevelSimilarityCalculator
from semantic_similarity.data_model import Phenotype
tb_cal = NodeLevelSimilarityCalculator()

def calculate_score(p, d):
    node_level = {}
    node_level[p.id] = {}
    p_syms = {Phenotype(id_, name) for id_, name in zip(p.hpos.id2hpo.keys(), p.hpos.name2hpo.keys())}
    d_syms = {Phenotype(id_, name) for id_, name in zip(d.hpos.id2hpo.keys(), d.hpos.name2hpo.keys())}
    score = tb_cal.get_semantic_similarity(p_syms, d_syms)
    return score

base_result = np.zeros(len(benchmark_patients), len(omim_diseases))
for paitent_idx, patient in tqdm.tqdm(enumerate(benchmark_dataset)):
    for disease_idx, disease in enumerate(omim_diseases.diseases):  ## 필요시 병렬처리
        score = calculate_score(patient, disease)
        base_result[patient_idx, disease_idx] = score

#### 비교 평가


아래의 np.ndarray을 이용하여 계산
- phen2disease_result(np.ndarray). shape=(# of patients, # of omim disease)
- lara_result(np.ndarray). shape=(# of patients, # of omim disease)
- base_result(np.ndarray). shape=(# of patients, # of omim disease)

In [ ]:
from tqdm import tqdm
from core_3asc.metric import topk_recall

result = []
for i, patient in tqdm(benchmark_patients):    
    result.append({
        "p_id": patient.id, 
        "scores_base": scores_base, 
        "scores_pd": scores_pd, 
        "scores_model": scores_model, 

        "top_1_base": topk_recall(scores_base, label, k=1),
        "top_1_pd": topk_recall(scores_pd, label, k=1),
        "top_1_model": topk_recall(scores_model, label, k=1),
        
        "top_5_base": topk_recall(scores_base, label, k=5),
        "top_5_pd": topk_recall(scores_pd, label, k=5),
        "top_5_model": topk_recall(scores_model, label, k=5),

        "top_10_base": topk_recall(scores_base, label, k=10),
        "top_10_pd": topk_recall(scores_pd, label, k=10),
        "top_10_model": topk_recall(scores_model, label, k=10),

        "top_15_base": topk_recall(scores_base, label, k=15),
        "top_15_pd": topk_recall(scores_pd, label, k=15),
        "top_15_model": topk_recall(scores_model, label, k=15),

        "top_20_base": topk_recall(scores_base, label, k=20),
        "top_20_pd": topk_recall(scores_pd, label, k=20),
        "top_20_model": topk_recall(scores_model, label, k=20),

        "top_30_base": topk_recall(scores_base, label, k=30),
        "top_30_pd": topk_recall(scores_pd, label, k=30),
        "top_30_model": topk_recall(scores_model, label, k=30),

        "top_40_base": topk_recall(scores_base, label, k=40),
        "top_40_pd": topk_recall(scores_pd, label, k=40),
        "top_40_model": topk_recall(scores_model, label, k=40),

        "top_50_base": topk_recall(scores_base, label, k=50),
        "top_50_pd": topk_recall(scores_pd, label, k=50),
        "top_50_model": topk_recall(scores_model, label, k=50),

        "top_75_base": topk_recall(scores_base, label, k=75),
        "top_75_pd": topk_recall(scores_pd, label, k=75),
        "top_75_model": topk_recall(scores_model, label, k=75),

        "top_100_base": topk_recall(scores_base, label, k=100),
        "top_100_pd": topk_recall(scores_pd, label, k=100),
        "top_100_model": topk_recall(scores_model, label, k=100),
        
    })

In [ ]:
base_data = {f"top{i}":None for i in [1,5,10,15,20,30,40,50,75,100]}
print(base_data)

In [ ]:
print(result)

In [ ]:
import pandas as pd
result_df = pd.DataFrame(result)
result_df = result_df.set_index("p_id")
result_df = result_df[list(set(result_df.columns) - {'p_id', 'scores_base', 'scores_pd', 'scores_model'})]

print(result_df)

data = (result_df.sum(0) / len(result_df)).to_dict()
print(data)

print(scores_base)

# Initialize empty dictionaries for 'base', 'pd', and 'model' data
# base_data = {'top1': None, 'top10': None, 'top50': None, 'top100': None}
# pd_data = {'top1': None, 'top10': None, 'top50': None, 'top100': None}
# model_data = {'top1': None, 'top10': None, 'top50': None, 'top100': None}

base_data = {f"top{i}": None for i in [1,5,10,15,20,30,40,50,75,100]}
pd_data = {f"top{i}": None for i in [1,5,10,15,20,30,40,50,75,100]}
model_data = {f"top{i}": None for i in [1,5,10,15,20,30,40,50,75,100]}

# Organize the data according to 'base', 'pd', and 'model' categories
for key, value in data.items():
    if 'base' in key:
        base_data[f"top{key.split('_')[1]}"] = value
    elif 'pd' in key:
        pd_data[f"top{key.split('_')[1]}"] = value
    elif 'model' in key:
        model_data[f"top{key.split('_')[1]}"] = value

# Create a DataFrame with 'base', 'pd', and 'model' as rows and 'top1', 'top10', 'top50', 'top100' as columns
df = pd.DataFrame([base_data, pd_data, model_data], index=['baseline', 'Pheno2Disease', 'LLM-based'])
df.index.name = 'Method'

# Display the DataFrame
print(df)

In [ ]:
# Figure 3

import matplotlib.pyplot as plt
import pandas as pd

fig_df = pd.DataFrame([base_data, pd_data, model_data], index=['Resnik-based IC', 'Pheno2Disease', 'LaRa'])
fig_df.index.name = 'Method'


# Example DataFrame
# Plotting the recall curve
plt.figure(figsize=(6, 6))
for index, row in fig_df.iterrows():
    plt.plot(list(row.index), list(row.values), marker='o', label=index)

plt.title('Real world dataset: rare disease patient data')
plt.xlabel('Top-k')
plt.ylabel('Top-k Recall')
plt.xticks(rotation=45)
plt.legend()
plt.grid(False)
plt.tight_layout()
plt.show()